# predict

> Common operations around the core datastructures for running a sim

In [ ]:
#| default_exp predict

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from flax.struct import dataclass as flax_dataclass
import jax.numpy as jnp
from typing import Optional
from chewc.population import Population

In [ ]:
#| export
"""
GBLUP (Genomic Best Linear Unbiased Prediction) implementation for chewc library.

This module provides functions for genomic prediction using the GBLUP methodology,
which is a standard approach in genomic selection and animal breeding.
"""

from typing import Optional, Dict
import jax
import jax.numpy as jnp
from jax.numpy.linalg import solve, inv, pinv
from flax.struct import dataclass as flax_dataclass

from chewc.population import Population

@flax_dataclass(frozen=True)
class PredictionResults:
    """
    A container for the results of a genomic prediction.
    """
    ids: jnp.ndarray
    ebv: jnp.ndarray
    pev: Optional[jnp.ndarray] = None
    reliability: Optional[jnp.ndarray] = None
    fixed_effects: Optional[jnp.ndarray] = None
    h2_used: Optional[float] = None
    var_components: Optional[Dict] = None



In [ ]:
#| export

@jax.jit
def _calculate_genomic_relationship_matrix(geno_dosage: jnp.ndarray) -> jnp.ndarray:
    """Calculate the genomic relationship matrix (G-matrix)."""
    n_ind, n_markers = geno_dosage.shape
    
    # Calculate allele frequencies
    p = jnp.mean(geno_dosage, axis=0) / 2.0
    
    # Center genotypes
    P = 2 * p
    Z = geno_dosage - P
    
    # Calculate G matrix using VanRaden method 1
    denominator = 2 * jnp.sum(p * (1 - p))
    G = (Z @ Z.T) / denominator
    
    # Add small regularization to ensure positive definite
    epsilon = 1e-6
    G_reg = G + jnp.identity(n_ind) * epsilon
    
    return G_reg

@jax.jit
def _solve_mixed_model_equations(
    X: jnp.ndarray,
    Z: jnp.ndarray, 
    y: jnp.ndarray,
    G_inv: jnp.ndarray,
    alpha: float
) -> tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """Solve mixed model equations for GBLUP."""
    n_ind = G_inv.shape[0]
    
    # Left-hand side matrix
    XTX = X.T @ X
    XTZ = X.T @ Z
    ZTX = Z.T @ X
    ZTZ_plus_alpha_Ginv = Z.T @ Z + alpha * G_inv
    
    lhs_top = jnp.hstack([XTX, XTZ])
    lhs_bottom = jnp.hstack([ZTX, ZTZ_plus_alpha_Ginv])
    lhs = jnp.vstack([lhs_top, lhs_bottom])
    
    # Right-hand side
    XTy = X.T @ y
    ZTy = Z.T @ y
    rhs = jnp.concatenate([XTy.flatten(), ZTy.flatten()])
    
    # Solve system
    solutions = jnp.linalg.solve(lhs, rhs)
    
    # Extract solutions
    n_fixed = X.shape[1]
    fixed_effects = solutions[:n_fixed]
    random_effects = solutions[n_fixed:]
    
    # Calculate coefficient matrix inverse for PEV
    C_inv = jnp.linalg.inv(lhs)
    
    return fixed_effects, random_effects, C_inv
# In chewc/predict.py

# ... (keep the existing _calculate_genomic_relationship_matrix and _solve_mixed_model_equations) ...
def gblup_predict(
    pop: Population, 
    h2: float = 0.5, 
    trait_idx: int = 0
) -> PredictionResults:
    """
    Perform GBLUP prediction, correctly handling a mix of individuals with
    and without phenotypes for model training and prediction.
    
    Args:
        pop: Population object. Can contain individuals with phenotypes (training set)
             and individuals with NaN phenotypes (prediction set).
        h2: Heritability.
        trait_idx: Index of the trait to predict.
        
    Returns:
        PredictionResults object with EBVs for ALL individuals.
    """
    dosage = pop.dosage
    pheno = pop.pheno[:, trait_idx:trait_idx+1]
    
    # Identify training (phenotyped) and prediction (unphenotyped) sets
    train_mask = ~jnp.isnan(pheno.flatten())
    pred_mask = jnp.isnan(pheno.flatten())
    
    n_train = jnp.sum(train_mask)
    n_pred = jnp.sum(pred_mask)
    
    if n_train == 0:
        raise ValueError("No valid phenotypes found for model training.")

    # --- Step 1: Calculate the full G-matrix for all individuals ---
    G_full = _calculate_genomic_relationship_matrix(dosage)
    
    # --- Step 2: Partition the G-matrix ---
    # JAX's jnp.ix_ does not support boolean masks.
    # Convert boolean masks to integer indices first.
    train_indices = jnp.where(train_mask)[0]
    
    # Now use the integer indices to slice the G-matrix.
    G_train_train = G_full[jnp.ix_(train_indices, train_indices)]
    
    # --- Step 3: Solve MME for the training set only ---
    dosage_train = dosage[train_mask] # Direct boolean masking is fine for 1D/2D arrays
    pheno_train = pheno[train_mask]
    
    G_train_inv = jnp.linalg.inv(G_train_train)
    
    X_train = jnp.ones((n_train, 1))
    Z_train = jnp.identity(n_train)
    alpha = (1.0 - h2) / h2
    
    fixed_eff, ebv_train, C_inv = _solve_mixed_model_equations(
        X_train, Z_train, pheno_train, G_train_inv, alpha
    )
    
    # --- Step 4: Predict EBVs for the prediction set (if any) ---
    all_ebv = jnp.zeros((pop.nInd, 1))
    all_ebv = all_ebv.at[train_mask, 0].set(ebv_train)

    if n_pred > 0:
        # Convert prediction mask to indices as well for consistency
        pred_indices = jnp.where(pred_mask)[0]
        G_pred_train = G_full[jnp.ix_(pred_indices, train_indices)]
        
        # The prediction formula: GEBV_pred = G_pred_train @ G_train_inv @ GEBV_train
        ebv_pred = G_pred_train @ G_train_inv @ ebv_train
        all_ebv = all_ebv.at[pred_mask, 0].set(ebv_pred)

    # --- Step 5: Calculate PEV and Reliability (for training set only) ---
    var_p = jnp.var(pheno_train)
    var_a = var_p * h2
    var_e = var_p * (1 - h2)
    
    n_fixed = X_train.shape[1]
    C22_inv = C_inv[n_fixed:, n_fixed:]
    pev_train = jnp.diag(C22_inv) * var_e
    reliability_train = jnp.maximum(0, 1 - (pev_train / var_a))
    
    all_pev = jnp.full((pop.nInd,), jnp.nan)
    all_reliability = jnp.full((pop.nInd,), jnp.nan)
    all_pev = all_pev.at[train_mask].set(pev_train)
    all_reliability = all_reliability.at[train_mask].set(reliability_train)
    
    return PredictionResults(
        ids=pop.id,
        ebv=all_ebv,
        pev=all_pev,
        reliability=all_reliability,
        fixed_effects=fixed_eff,
        h2_used=h2
    )


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()